In [1]:
import os
import math
import datetime
import numpy as np
import pandas as pd
from io import StringIO

In [2]:
block = """
Person,28800.000000,1,0,7,75.000000,80,1.000000,1.000000,28842.000000,6,1,28800.000000,28840.100000,2,7,1,0,0,0,0,0,0,2.000000,0.000000,10.000000,0,1
Person,28801.000000,1,0,3,75.000000,80,1.000000,1.000000,28840.200000,6,2,28801.100000,28838.300000,2,3,1,0,1,0,0,0,0,2.000000,0.000000,10.000000,0,1
Person,28802.000000,1,0,6,75.000000,80,1.000000,1.000000,28846.700000,6,3,28802.100000,28844.800000,2,6,1,0,2,0,0,0,0,4.200000,0.000000,10.000000,0,1
Person,28803.000000,1,0,7,75.000000,80,1.000000,1.000000,28876.200000,6,4,28803.100000,28874.300000,2,7,1,0,3,0,0,0,0,8.100000,0.000000,10.000000,0,1
Person,28803.000000,1,0,8,75.000000,80,1.000000,1.000000,28845.900000,6,5,28803.200000,28844.000000,2,8,1,0,4,0,0,0,0,2.000000,0.000000,10.000000,0,1
Person,28804.000000,1,0,2,75.000000,80,1.000000,1.000000,28847.700000,6,6,28804.100000,28845.800000,2,2,1,0,5,0,0,0,0,3.900000,0.000000,10.000000,0,1
Person,28804.000000,1,0,9,75.000000,80,1.000000,1.000000,28847.700000,6,7,28804.200000,28845.800000,2,9,1,0,6,0,0,0,0,2.000000,0.000000,10.000000,0,1
Person,28805.000000,1,0,9,75.000000,80,1.000000,1.000000,28848.800000,6,7,28804.200000,28845.800000,2,9,1,0,7,0,0,0,0,2.000000,0.000000,10.000000,0,1
Person,28807.000000,1,0,2,75.000000,80,1.000000,1.000000,28848.800000,6,6,28804.100000,28845.800000,2,2,1,0,8,0,0,0,0,3.800000,0.000000,10.000000,0,1
Person,28807.000000,1,0,6,75.000000,80,1.000000,1.000000,28847.800000,6,3,28802.100000,28844.800000,2,6,1,0,9,0,0,0,0,2.000000,0.000000,10.000000,0,1
"""
block = pd.read_csv(StringIO(block), header=None)
block

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,Person,28800.0,1,0,7,75.0,80,1.0,1.0,28842.0,...,0,0,0,0,0,2.0,0.0,10.0,0,1
1,Person,28801.0,1,0,3,75.0,80,1.0,1.0,28840.2,...,1,0,0,0,0,2.0,0.0,10.0,0,1
2,Person,28802.0,1,0,6,75.0,80,1.0,1.0,28846.7,...,2,0,0,0,0,4.2,0.0,10.0,0,1
3,Person,28803.0,1,0,7,75.0,80,1.0,1.0,28876.2,...,3,0,0,0,0,8.1,0.0,10.0,0,1
4,Person,28803.0,1,0,8,75.0,80,1.0,1.0,28845.9,...,4,0,0,0,0,2.0,0.0,10.0,0,1
5,Person,28804.0,1,0,2,75.0,80,1.0,1.0,28847.7,...,5,0,0,0,0,3.9,0.0,10.0,0,1
6,Person,28804.0,1,0,9,75.0,80,1.0,1.0,28847.7,...,6,0,0,0,0,2.0,0.0,10.0,0,1
7,Person,28805.0,1,0,9,75.0,80,1.0,1.0,28848.8,...,7,0,0,0,0,2.0,0.0,10.0,0,1
8,Person,28807.0,1,0,2,75.0,80,1.0,1.0,28848.8,...,8,0,0,0,0,3.8,0.0,10.0,0,1
9,Person,28807.0,1,0,6,75.0,80,1.0,1.0,28847.8,...,9,0,0,0,0,2.0,0.0,10.0,0,1


## Parse ELVR

#### 0.1 Parse .elvr to Initial Logs

In [7]:
def parse_elvr(filepath, print_log = False) -> list[dict]:
    # ---- Pasrse ELVR ----
    logs = []
    with open(filepath, "r", encoding="utf-8") as f:
        cur_table_category = None
        cur_table = []
        run = None
        sim_id = None
        lift_count = None
        field_mapping = {
            "SpatialPlot": ["lift_id", "time", "lobby_id", "load", "area"],
            "Person": ["time_arrived", "lobby_id", "unk_index_1", "destination_id", "weight","capacity_factor", "loading_time", "unloading_time", "tbc_time_disembarked",
                       "unk_index_2", "lift_id", "tbc_wait_time_end", "tbc_transit_time_end", 
                       "tbc_index_3", "actual_destination_id", "tbc_index_4", "tbc_index_5", "tbc_index_6", "tbc_index_7", "tbc_index_8", "tbc_index_9", "tbc_index_10", 
                       "tbc_metric_11", "tbc_metric_12", "tbc_metric_13", "tbc_index_14", "tbc_index_15"],
            }
        time_start = datetime.datetime.now()
        skip_categories = ["NoPassengers", "RemoteMonitoring"]

        # ---- Parse through each line ----
        for line in f: 
            # ---- Clean and Skip blank lines ----
            line = line.strip()
            if not line: continue
            # ---- Get Summary Inforamtion ----
            if line.startswith("SimulationID"):
                cells = line.split(",")
                sim_id = cells[0].split(": ")[1]
                run = cells[1].strip()
                if print_log: print("Loading Simulation ID: " + sim_id + ", Run: " + run)
                continue
            
            # ---- Collect Table Information ----
            entry_category = line.split(",")[0]
            
            # ---- Log First Line ----
            if cur_table_category is None:
                cur_table_category = entry_category
                cur_table.append(line)
                continue

            # ---- Log End of Table ----
            if (cur_table_category != entry_category): # End of current table
                if (cur_table_category not in skip_categories):
                    # ---- Log and Clear Table ----
                    if print_log: print(f"Loading Dataframe: {cur_table_category} with {len(cur_table)} entries")
                    df_elvr = pd.read_csv(StringIO("\n".join(cur_table)), header=None, low_memory=True)
                    df_elvr.drop(df_elvr.columns[0], axis=1, inplace=True)
                    # ---- Apply Field Mapping ----
                    if cur_table_category in field_mapping: df_elvr.columns = field_mapping[cur_table_category]
                    # ---- Get Lift Count ----
                    if cur_table_category == "SpatialPlot": lift_count = df_elvr.iloc[:, 0].nunique()
                    logs.append({"category": cur_table_category, "dataframe": df_elvr, "simulation_id": sim_id, "run": run, "lift_count": lift_count})
                    cur_table = []

                # ---- Start New Table ----
                cur_table_category = entry_category
                # ---- Skip Lines based on categories ----
                if cur_table_category in skip_categories: continue
                cur_table.append(line)
                continue
            
            # ---- Log Lines ----
            if cur_table_category == entry_category: # Collect Rows
                cur_table.append(line)
                continue
            

    print(f"Finished Parsing {os.path.basename(filepath)} \nEntries Logged: {len(logs)} \nProcessing Time: {(datetime.datetime.now() - time_start).total_seconds()}s")

    return logs

In [8]:
directory = r"C:\Users\tichen\OneDrive - Foster + Partners\Documents\tc_archive\Py_Project\Elevate\Resource\elevate\OneDrive_1_23-06-2025\Local Lifts - Elevate Files\3 Zone - 100 Uppeak -SD"
file_name = r"North Tower - Office - High Zone.elvr"
# ---- Locate CSV ----
filepath = os.path.join(directory, file_name)
elvr_logs = parse_elvr(filepath, print_log=True)

df_lift_elvr = elvr_logs[0]["dataframe"]
df_passenger_elvr = elvr_logs[1]["dataframe"]

Loading Simulation ID: 847, Run: 1
Loading Dataframe: SpatialPlot with 1837 entries
Loading Dataframe: Person with 3593 entries
Loading Simulation ID: 847, Run: 2
Loading Dataframe: SpatialPlot with 2745 entries
Loading Dataframe: Person with 3592 entries
Loading Simulation ID: 847, Run: 3
Loading Dataframe: SpatialPlot with 1887 entries
Loading Dataframe: Person with 3593 entries
Loading Simulation ID: 847, Run: 4
Loading Dataframe: SpatialPlot with 1847 entries
Loading Dataframe: Person with 3593 entries
Loading Simulation ID: 847, Run: 5
Loading Dataframe: SpatialPlot with 2073 entries
Loading Dataframe: Person with 3592 entries
Loading Simulation ID: 847, Run: 6
Loading Dataframe: SpatialPlot with 1845 entries
Loading Dataframe: Person with 3593 entries
Loading Simulation ID: 847, Run: 7
Loading Dataframe: SpatialPlot with 1943 entries
Loading Dataframe: Person with 3593 entries
Loading Simulation ID: 847, Run: 8
Loading Dataframe: SpatialPlot with 2059 entries
Loading Dataframe: P

#### 0.2 Parse ELVR Logs Level 1
Create a Dictionary for each run of Lift based on Lift ID & Passenger Logs based on Lobby IDs

In [9]:
def parse_lift_elvr(df_lift_elvr:pd.DataFrame) -> dict[str, pd.DataFrame]:
    # ---- Fetch Passenger Dataframe ----
    df_lift = df_lift_elvr[["lift_id", "lobby_id", "time", "load", "area"]].copy()
    df_lift_dict = {}
    # ---- Log Lift Status ----
    for lift_id, df_lift_split in df_lift.groupby("lift_id"):
        # ---- Sort by Time and Reset Index ----
        lift_logbook = df_lift_split.copy()
        lift_logbook = lift_logbook.sort_values(by="time").reset_index(drop=True)

        status_series = []
        for log in lift_logbook.itertuples(index=True):
            # ---- Initialize Variables ----
            status = "unknown"
            # ---- Even Index: Arrival ----
            if log.Index % 2 == 0: 
                status = "arriving"                
            # ---- Odd Index: Departure ----
            else:
                if log.Index + 1 < len(lift_logbook):  # Check if further depature log exists
                    cur_lobby_id = log.lobby_id
                    next_lobby_id = lift_logbook.at[log.Index+1, "lobby_id"]
                    if next_lobby_id > cur_lobby_id:
                        status = "ascending"
                    else:
                        status = "descending"
                # ---- Last Index: Termination ----
                else: 
                    status = "terminating"
            status_series.append(status)
        lift_logbook.insert(3, "status", status_series, allow_duplicates=False)

        # ---- Add to Dictionary ----
        df_lift_dict[lift_id] = lift_logbook

    df_lift = pd.concat(list(df_lift_dict.values()), axis=0).reset_index(drop=True)

    return df_lift

In [10]:
def parse_passenger_elvr(df_passenger_elvr:pd.DataFrame) -> dict[str, pd.DataFrame]:
    # ---- Fetch Passenger Dataframe ----
    df_passenger = df_passenger_elvr[["lobby_id", "destination_id", "lift_id", "time_arrived", "tbc_wait_time_end", "tbc_transit_time_end", "tbc_time_disembarked"]].copy()
    df_passenger.insert(0, "passenger_id", range(len(df_passenger)), allow_duplicates=False)
    # ---- Calculate KPI Metrics Part 1 ----
    adjusted_wait_time_end = df_passenger[['tbc_wait_time_end', 'time_arrived']].astype(float).max(axis=1)
    df_passenger['wait_time'] = adjusted_wait_time_end - df_passenger['time_arrived']
    df_passenger['transit_time'] = df_passenger['tbc_transit_time_end'] - adjusted_wait_time_end
    df_passenger['travel_time'] = df_passenger['wait_time'] + df_passenger['transit_time']

    return df_passenger

In [11]:
lift_logbook_l1 = parse_lift_elvr(df_lift_elvr)
passenger_logbook_l1 = parse_passenger_elvr(df_passenger_elvr)
lift_logbook_l1.tail(10)

,lift_id,lobby_id,time,status,load,area
1827,7,4,33298.2,ascending,375.0,1.05
1828,7,6,33305.3,arriving,375.0,1.05
1829,7,6,33315.6,ascending,150.0,0.42
1830,7,8,33322.7,arriving,150.0,0.42
1831,7,8,33331.9,descending,0.0,0.00
1832,7,1,33364.6,arriving,0.0,0.00
1833,7,1,33373.8,ascending,150.0,0.42
1834,7,5,33404.4,arriving,150.0,0.42
1835,7,5,33412.5,ascending,75.0,0.21
1836,7,8,33420.9,arriving,75.0,0.21


#### 0.3 Generate Timeline

In [12]:
def get_timeline_logbooks_slow(passenger_logbook: pd.DataFrame) -> dict:
    timeline_logbooks = {}

    for lobby_id, df_passenger_split in passenger_logbook.groupby("lobby_id"):
        # ---- Initialize Timeline ----
        start_time = math.floor(df_passenger_split['time_arrived'].min())
        finish_time = math.ceil(df_passenger_split['tbc_time_disembarked'].max())
        df_timeline = pd.DataFrame({'time': range(start_time, finish_time)})

        # ---- Initialize KPI Metrics ----
        passenger_id_register_series = []
        wait_time_register_series = []
        transit_time_register_series = []
        travel_time_register_series = []

        queue_length_series = []
        mean_wait_time_series = []
        max_wait_time_series = []
        mean_transit_time_series = []
        max_transit_time_series = []
        mean_travel_time_series = []
        max_travel_time_series = []
        # ---- Count Wait Time for each Passenger at each second ----
        # df_passenger['time_range_id'] = pd.cut(df_passenger['wait_time_end_ref'], bins=10, labels=False)
        pas_wait_time_start = df_passenger_split['time_arrived']
        pas_wait_time_end = df_passenger_split[['tbc_wait_time_end', 'time_arrived']].max(axis=1)
        for log in df_timeline.itertuples(index=False):
            # ---- Collect Passenger Logs ----
            time = log.time
            passenger_resgister = df_passenger_split[(pas_wait_time_start <= time) & (pas_wait_time_end >= time)]
            # ---- Register Passenger Information ----
            passenger_id_register_series.append(passenger_resgister['passenger_id'].tolist())
            wait_time_register_series.append(passenger_resgister['wait_time'].tolist())
            transit_time_register_series.append(passenger_resgister['transit_time'].tolist())
            travel_time_register_series.append(passenger_resgister['travel_time'].tolist())
            # ---- Get KPIs ----
            queue_length_series.append(len(passenger_resgister))
            mean_wait_time_series.append(passenger_resgister['wait_time'].mean() if not passenger_resgister.empty else 0)
            max_wait_time_series.append(passenger_resgister['wait_time'].max() if not passenger_resgister.empty else 0)
            mean_transit_time_series.append(passenger_resgister['transit_time'].mean() if not passenger_resgister.empty else 0)
            max_transit_time_series.append(passenger_resgister['transit_time'].max() if not passenger_resgister.empty else 0)
            mean_travel_time_series.append(passenger_resgister['travel_time'].mean() if not passenger_resgister.empty else 0)
            max_travel_time_series.append(passenger_resgister['travel_time'].max() if not passenger_resgister.empty else 0)
            
        # ---- Add KPI Metrics to Timeline ----
        df_timeline['passenger_register'] = passenger_id_register_series
        df_timeline['queue_length'] = queue_length_series
        df_timeline['mean_wait_time'] = mean_wait_time_series
        df_timeline['mean_transit_time'] = mean_transit_time_series
        df_timeline['mean_travel_time'] = mean_travel_time_series

        df_timeline['max_wait_time'] = max_wait_time_series
        df_timeline['max_transit_time'] = max_transit_time_series
        df_timeline['max_travel_time'] = max_travel_time_series

        df_timeline['wait_time_register'] = wait_time_register_series
        df_timeline['transit_time_register'] = transit_time_register_series
        df_timeline['travel_time_register'] = travel_time_register_series
        
        timeline_logbooks[lobby_id] = df_timeline

    return timeline_logbooks

In [8]:
def get_timeline_logbooks(passenger_logbook: pd.DataFrame) -> dict:
    timeline_logbooks = {}

    for lobby_id, df_passenger_split in passenger_logbook.groupby("lobby_id"):
        start_time = math.floor(df_passenger_split['time_arrived'].min())
        finish_time = math.ceil(df_passenger_split['tbc_time_disembarked'].max())
        times = np.arange(start_time, finish_time)

        # Prepare intervals for each passenger (wait interval)
        pas_wait_time_start = df_passenger_split['time_arrived'].values
        pas_wait_time_end = df_passenger_split[['tbc_wait_time_end', 'time_arrived']].max(axis=1).values

        # Build a 2D boolean mask: rows are times, columns are passengers
        # mask[t, p] = True if passenger p is waiting at time t
        mask = (times[:, None] >= pas_wait_time_start[None, :]) & (times[:, None] <= pas_wait_time_end[None, :])

        # Pre-fetch required columns as arrays for fast slicing
        passenger_ids = df_passenger_split['passenger_id'].values
        wait_times = df_passenger_split['wait_time'].values
        transit_times = df_passenger_split['transit_time'].values
        travel_times = df_passenger_split['travel_time'].values

        # For each time, get list of passenger IDs and KPIs (vectorized)
        #passenger_id_register_series = [passenger_ids[row].tolist() for row in mask.T.nonzero()[1].reshape(mask.shape[0], -1)]
        # Alternative (and faster): build with list comprehensions
        passenger_id_register_series = [passenger_ids[mask[i]].tolist() for i in range(len(times))]
        wait_time_register_series = [wait_times[mask[i]].tolist() for i in range(len(times))]
        transit_time_register_series = [transit_times[mask[i]].tolist() for i in range(len(times))]
        travel_time_register_series = [travel_times[mask[i]].tolist() for i in range(len(times))]

        # KPIs (vectorized)
        queue_length_series = mask.sum(axis=1)
        def safe_stat(arr, func, empty_val=0):
            return func(arr) if len(arr) else empty_val
        mean_wait_time_series = [np.mean(wait_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]
        max_wait_time_series = [np.max(wait_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]
        mean_transit_time_series = [np.mean(transit_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]
        max_transit_time_series = [np.max(transit_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]
        mean_travel_time_series = [np.mean(travel_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]
        max_travel_time_series = [np.max(travel_times[mask[i]]) if queue_length_series[i] else 0 for i in range(len(times))]

        # Build DataFrame
        df_timeline = pd.DataFrame({
            'time': times,
            'passenger_register': passenger_id_register_series,
            'queue_length': queue_length_series,
            'mean_wait_time': mean_wait_time_series,
            'max_wait_time': max_wait_time_series,
            'mean_transit_time': mean_transit_time_series,
            'max_transit_time': max_transit_time_series,
            'mean_travel_time': mean_travel_time_series,
            'max_travel_time': max_travel_time_series,
            'wait_time_register': wait_time_register_series,
            'transit_time_register': transit_time_register_series,
            'travel_time_register': travel_time_register_series,
        })

        timeline_logbooks[lobby_id] = df_timeline

    return timeline_logbooks

0.3.2 Compile Runs

In [11]:
def compile_timeline(df_timelines:list[pd.DataFrame]) -> pd.DataFrame:
    '''
    Compile timeline logbooks across multiple lobbys and/or runs
    '''
    # ---- Aggregate Timeline Dataframes ----
    df_compiled = pd.concat(df_timelines).groupby('time', as_index=False).agg(list)
    # ---- Drop Fields ----
    df_compiled.drop(columns=['passenger_register'], inplace=True)

    # ---- Compile KPI Metrics ----
    # ---- Insert Fields to Passenger Dataframe ----
    df_compiled['queue_length_regiester'] = df_compiled['queue_length']
    df_compiled['queue_length'] = df_compiled['queue_length'].apply(lambda x: max(x) if len(x)!= 0 else 0)

    df_compiled['wait_time_register'] = df_compiled['wait_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    df_compiled['transit_time_register'] = df_compiled['transit_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    df_compiled['travel_time_register'] = df_compiled['travel_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    
    df_compiled['mean_wait_time_register'] = df_compiled['mean_wait_time']
    df_compiled['mean_transit_time_register'] = df_compiled['mean_transit_time']
    df_compiled['mean_travel_time_register'] = df_compiled['mean_travel_time']

    df_compiled['mean_wait_time'] = df_compiled['wait_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)
    df_compiled['mean_transit_time'] = df_compiled['transit_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)
    df_compiled['mean_travel_time'] = df_compiled['travel_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)

    df_compiled['max_wait_time'] = df_compiled['max_wait_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)
    df_compiled['max_transit_time'] = df_compiled['max_transit_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)
    df_compiled['max_travel_time'] = df_compiled['max_travel_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)

    return df_compiled

In [70]:
timeline_logbooks = get_timeline_logbooks(passenger_logbook_l1)
timeline_logbook = timeline_logbooks[next(iter(timeline_logbooks))]

In [73]:
timeline_logbook

,time,passenger_register,queue_length,mean_wait_time,max_wait_time,mean_transit_time,max_transit_time,mean_travel_time,max_travel_time,wait_time_register,transit_time_register,travel_time_register
0,28800,[0],1,0.00,0.0,40.10,40.1,40.10,40.1,[0.0],[40.099999999998545],[40.099999999998545]
1,28801,[1],1,0.10,0.1,37.20,37.2,37.30,37.3,[0.09999999999854481],[37.20000000000073],[37.29999999999927]
2,28802,[2],1,0.10,0.1,42.70,42.7,42.80,42.8,[0.09999999999854481],[42.70000000000073],[42.79999999999927]
3,28803,"[3, 4]",2,0.15,0.2,56.00,71.2,56.15,71.3,"[0.09999999999854481, 0.2000000000007276]","[71.20000000000073, 40.79999999999927]","[71.29999999999927, 41.0]"
4,28804,"[5, 6]",2,0.15,0.2,41.65,41.7,41.80,41.8,"[0.09999999999854481, 0.2000000000007276]","[41.70000000000073, 41.599999999998545]","[41.79999999999927, 41.79999999999927]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4618,33418,[],0,0.00,0.0,0.00,0.0,0.00,0.0,[],[],[]
4619,33419,[],0,0.00,0.0,0.00,0.0,0.00,0.0,[],[],[]
4620,33420,[],0,0.00,0.0,0.00,0.0,0.00,0.0,[],[],[]
4621,33421,[],0,0.00,0.0,0.00,0.0,0.00,0.0,[],[],[]


#### 0.4 Compile Runs

#### 0.4 Cross Reference Logbooks (Lift & Passengers)

In [12]:
def cross_reference_logbook_passenger(lift_logbooks_l1:dict, passenger_logbooks_l1:dict):
    # ---- Cross Reference Logbooks ----
    for lobby_id, df_passenger in passenger_logbooks_l1.items():
        # df_passenger is a reference not a copy
        # ---- Split Passengers by Lift Used ----
        for lift_id, df_passenger_split in df_passenger.groupby("lift_id"):
            # Groupby creates copies not references
            # ---- Get Lift Logs for the Specific Lift and Lobby ----
            df_lift = lift_logbooks_l1[lift_id]
            df_lift_lobby_match = df_lift[df_lift["lobby_id"] == lobby_id]
            df_lift_depart = df_lift_lobby_match[df_lift_lobby_match["status"] != "arriving"]

            # ---- Initialie New Fields for Passenger Logbook ----
            lift_depart_log_index_series = []
            lift_rea_desti_log_index_series = []
            time_lift_departed_series = []
            time_lift_arrived_series = []
            time_lift_rea_desti_series = []

            # ---- Loop through each Passenger Log ----
            for passenger_log in df_passenger_split.itertuples(index=False):
                # ---- Find the Lift Log marking Departure AFTER Passenger Arrival/Wait-Time-End & BEFORE Passenger Tranzit-Time-End  ----
                wait_time_end = max(passenger_log.tbc_wait_time_end, passenger_log.time_arrived)
                log_lift_depart = df_lift_depart[df_lift_depart["time"] >= wait_time_end]
                lift_depart_log_index = log_lift_depart.index[0] 
                lift_arrive_log_index = lift_depart_log_index - 1
                
                time_lift_departed = log_lift_depart.at[lift_depart_log_index, "time"] 
                time_lift_arrived = df_lift.at[lift_arrive_log_index, "time"] 

                lift_depart_log_index_series.append(lift_depart_log_index)
                time_lift_departed_series.append(time_lift_departed)
                time_lift_arrived_series.append(time_lift_arrived)

                # ---- Find Desitation Log ----
                lift_rea_desti_log_index = lift_arrive_log_index + 1 if lift_arrive_log_index + 1 < len(df_lift) else None
                time_lift_rea_desti = None
                if lift_rea_desti_log_index is None: continue
                for i in range(lift_rea_desti_log_index, len(df_lift)):
                    if df_lift.at[i, "status"] != "arriving": continue
                    if df_lift.at[i, "lobby_id"] == passenger_log.destination_id:
                        lift_rea_desti_log_index = i
                        time_lift_rea_desti = df_lift.at[i, "time"]
                        break
                lift_rea_desti_log_index_series.append(lift_rea_desti_log_index)
                time_lift_rea_desti_series.append(time_lift_rea_desti)
            
            # ---- Add Data to Passenger Logbook ----
            df_passenger.loc[df_passenger_split.index, "lift_depart_log_index"] = lift_depart_log_index_series
            df_passenger.loc[df_passenger_split.index, "lift_rea_desti_log_index"] = lift_rea_desti_log_index_series
            df_passenger.loc[df_passenger_split.index, "time_lift_departed"] = time_lift_departed_series
            df_passenger.loc[df_passenger_split.index, "time_lift_arrived"] = time_lift_arrived_series
            df_passenger.loc[df_passenger_split.index, "time_lift_reached_destination"] = time_lift_rea_desti_series
    
    return passenger_logbooks_l1

In [13]:
def cross_reference_logbooks(lift_logbooks_l1:dict, passenger_logbooks_l1:dict):
    # ---- Create Deep Copies ---- 
    lift_logbooks_l2 = {key: value.copy() for key, value in lift_logbooks_l1.items()}
    passenger_logbooks_l2 = {key: value.copy() for key, value in passenger_logbooks_l1.items()}
    # ---- Cross Reference Logbooks ----
    for lobby_id, df_passenger in passenger_logbooks_l2.items():
        # df_passenger is a reference not a copy
        # ---- Split Passengers by Lift Used ----
        for lift_id, df_passenger_split in df_passenger.groupby("lift_id"):
            # Groupby creates copies not references
            # ---- Get Lift Logs for the Specific Lift and Lobby ----
            df_lift = lift_logbooks_l2[lift_id]
            df_lift_lobby_match = df_lift[df_lift["lobby_id"] == lobby_id]
            df_lift_depart = df_lift_lobby_match[df_lift_lobby_match["status"] != "arriving"]

            # ---- Initialie New Fields for Passenger Logbook ----
            lift_depart_log_index_register = {}
            lift_rea_desti_log_index_register = {}
            lift_depart_log_index_series = []
            lift_rea_desti_log_index_series = []
            time_lift_departed_series = []
            time_lift_arrived_series = []
            time_lift_rea_desti_series = []

            # ---- Loop through each Passenger Log ----
            for passenger_log in df_passenger_split.itertuples(index=False):
                # ---- Find the Lift Log marking Departure AFTER Passenger Arrival/Wait-Time-End & BEFORE Passenger Tranzit-Time-End  ----
                wait_time_end = max(passenger_log.tbc_wait_time_end, passenger_log.time_arrived)
                log_lift_depart = df_lift_depart[df_lift_depart["time"] >= wait_time_end]
                lift_depart_log_index = log_lift_depart.index[0] 
                lift_arrive_log_index = lift_depart_log_index - 1
                
                time_lift_departed = log_lift_depart.at[lift_depart_log_index, "time"] 
                time_lift_arrived = df_lift.at[lift_arrive_log_index, "time"] 

                if lift_depart_log_index not in lift_depart_log_index_register:
                    lift_depart_log_index_register[lift_depart_log_index] = []
                lift_depart_log_index_register[lift_depart_log_index].append(passenger_log.passenger_id)
                lift_depart_log_index_series.append(lift_depart_log_index)
                time_lift_departed_series.append(time_lift_departed)
                time_lift_arrived_series.append(time_lift_arrived)

                # ---- Find Desitation Log ----
                lift_rea_desti_log_index = lift_arrive_log_index + 1 if lift_arrive_log_index + 1 < len(df_lift) else None
                time_lift_rea_desti = None
                if lift_rea_desti_log_index is None: continue
                for i in range(lift_rea_desti_log_index, len(df_lift)):
                    if df_lift.at[i, "status"] != "arriving": continue
                    if df_lift.at[i, "lobby_id"] == passenger_log.destination_id:
                        lift_rea_desti_log_index = i
                        time_lift_rea_desti = df_lift.at[i, "time"]
                        break
                if lift_rea_desti_log_index not in lift_rea_desti_log_index_register:
                    lift_rea_desti_log_index_register[lift_rea_desti_log_index] = []
                lift_rea_desti_log_index_register[lift_rea_desti_log_index].append(passenger_log.passenger_id)
                lift_rea_desti_log_index_series.append(lift_rea_desti_log_index)
                time_lift_rea_desti_series.append(time_lift_rea_desti)
            
            # ---- Add Data to Passenger Logbook ----
            df_passenger.loc[df_passenger_split.index, "lift_depart_log_index"] = lift_depart_log_index_series
            df_passenger.loc[df_passenger_split.index, "lift_rea_desti_log_index"] = lift_rea_desti_log_index_series
            df_passenger.loc[df_passenger_split.index, "time_lift_departed"] = time_lift_departed_series
            df_passenger.loc[df_passenger_split.index, "time_lift_arrived"] = time_lift_arrived_series
            df_passenger.loc[df_passenger_split.index, "time_lift_reached_destination"] = time_lift_rea_desti_series

            # ---- Add Data to Lift Logbook ----
            if "boarding_register" not in df_lift.columns: df_lift["boarding_register"] = None
            if "disembarking_register" not in df_lift.columns: df_lift["disembarking_register"] = None
            for (idx, passenger_register) in lift_depart_log_index_register.items():
                df_lift.at[int(idx), "boarding_register"] = passenger_register
            for (idx, passenger_register) in lift_rea_desti_log_index_register.items():
                df_lift.at[int(idx), "disembarking_register"] = passenger_register

    
    return lift_logbooks_l2, passenger_logbooks_l2

In [14]:
lift_logbooks_l2, passenger_logbooks_l2 = cross_reference_logbooks(lift_logbooks_l1, passenger_logbooks_l1)
lift_logbook_l2 = lift_logbooks_l2[next(iter(lift_logbooks_l1))]
passenger_logbook_l2 = passenger_logbooks_l2[next(iter(passenger_logbooks_l2))]
passenger_logbook_l2.tail(20)

,passenger_id,lobby_id,destination_id,lift_id,time_arrived,tbc_wait_time_end,tbc_transit_time_end,tbc_time_disembarked,wait_time,transit_time,travel_time,lift_depart_log_index,lift_rea_desti_log_index,time_lift_departed,time_lift_arrived,time_lift_reached_destination
3573,3573,1,2,3,33287.0,33314.0,33369.2,33379.9,27.0,55.2,82.2,263.0,264.0,33340.9,33314.1,33369.3
3574,3574,1,8,5,33288.0,33296.1,33355.6,33370.7,8.1,59.5,67.6,239.0,240.0,33323.0,33296.2,33355.7
3575,3577,1,3,2,33288.0,33373.4,33411.7,33413.6,85.4,38.3,123.7,257.0,258.0,33382.7,33373.5,33411.8
3576,3576,1,6,4,33288.0,33304.8,33356.3,33363.7,16.8,51.5,68.3,267.0,268.0,33325.1,33304.9,33356.4
3577,3575,1,7,6,33288.0,33312.5,33366.9,33375.4,24.5,54.4,78.9,263.0,264.0,33335.0,33312.6,33367.0
3578,3578,1,8,5,33289.0,33296.1,33355.6,33371.8,7.1,59.5,66.6,239.0,240.0,33323.0,33296.2,33355.7
3579,3579,1,5,1,33289.0,33304.2,33383.3,33389.6,15.2,79.1,94.3,255.0,258.0,33331.1,33304.3,33383.4
3580,3582,1,2,3,33291.0,33314.0,33369.2,33381.0,23.0,55.2,78.2,263.0,264.0,33340.9,33314.1,33369.3
3581,3581,1,8,5,33291.0,33296.1,33355.6,33372.9,5.1,59.5,64.6,239.0,240.0,33323.0,33296.2,33355.7
3582,3580,1,4,1,33291.0,33304.2,33360.9,33371.6,13.2,56.7,69.9,255.0,256.0,33331.1,33304.3,33361.0


#### xx

In [19]:
def get_elevator_logs_dict(df_lift_elvr, df_passenger_elvr, debug_log = False) -> dict:
    # ---- Fetch Passenger Dataframe ----
    df_lift = df_lift_elvr[["lift_id", "lobby_id", "time", "load", "area"]].copy()
    
    # ---- Create a dictionary of DataFrames for each lift_id ----
    lift_ids = df_lift["lift_id"].unique()
    df_lift_dict = {
        lift_id: df_lift[df_lift["lift_id"] == lift_id]
        .sort_values(by="time")
        .reset_index(drop=True)
        for lift_id in lift_ids
    }
    # ---- Create a dictionary of DataFrames for each lift_id ----
    df_pas_short = df_passenger_elvr[['lift_id', 'lobby_id', 'time_boarding', "time_destination_reached"]].copy()
    df_pas_short = df_pas_short.sort_values(['lift_id', 'lobby_id', 'time_boarding'])
    
    # ---- Iterate through each Lift Log ----
    for lift_id, df_lift_logs in df_lift_dict.items():
        pas_onboard_live = []
        status_series = []
        passengers_onboard_series = []
        passengers_boarded_series = []
        passengers_disembarked_series = []
        # Find only lift logs  where lobby_id and lift_id match the passenger's records
        df_pas_matched_lift = df_pas_short[df_pas_short["lift_id"] == lift_id]

        # ---- Fetch Passenger Dataframe for the current lift_id ----
        for log in df_lift_logs.itertuples(index=True):
            # ---- Initialize Variables ----
            status = "unknown"
            pas_boarding_list = []
            pas_disembarking_list = []

            # ---- Even Index: Arrival ----
            if log.Index % 2 == 0: 
                status = "arriving"
                if debug_log: print(f"passenger_onboard_on_arrival: {pas_onboard_live}")
                
            # ---- Odd Index: Departure ----
            else:
                # ---- Update Status ----
                if log.Index + 1 < len(df_lift_logs):  # Check if further depature log exists
                    cur_lobby_id = log.lobby_id
                    next_lobby_id = df_lift_logs.at[log.Index+1, "lobby_id"]
                    if next_lobby_id > cur_lobby_id:
                        status = "ascending"
                    else:
                        status = "descending"
                # ---- Last Index: Termination ----
                else: 
                    status = "terminating"
                
                # ---- Find who disembarked ----
                if pas_onboard_live:
                    df_pas_onboard = df_passenger_elvr.iloc[pas_onboard_live]
                    # ---- Filter only passenger logs that reachded destination same time lift arrived ----
                    pas_disembarking_list = df_pas_onboard[df_pas_onboard["time_disembarked"] < log.time].index.tolist()
                    if debug_log: print(f"disembarking: {pas_disembarking_list}")

                    # ---- Remove Passengers who disembarked ----
                    pas_onboard_live = [id for id in pas_onboard_live if id not in pas_disembarking_list]

                # ---- Find who boarded ----
                pas_boarding_list = df_pas_matched_lift[
                    # ---- Filter only passenger logs that boarded before lift departed, and reached destination after lift departed ----
                    (df_pas_matched_lift["lobby_id"] == log.lobby_id) &
                    (df_pas_matched_lift["time_boarding"] < log.time) & 
                    (df_pas_matched_lift["time_destination_reached"] > log.time)
                    ].index.tolist()
                if debug_log: print(f"boarding: {pas_boarding_list}")

                # ---- Add Passengers who boarded ----
                pas_onboard_live.extend(pas_boarding_list)
                if debug_log: print(f"passenger_onboard_on_departure: {pas_onboard_live}")
            # ---- Update Variables ----
            status_series.append(status)
            passengers_onboard_series.append(pas_onboard_live.copy())
            passengers_boarded_series.append(pas_boarding_list)
            passengers_disembarked_series.append(pas_disembarking_list)
        
        df_lift_logs.insert(3, "status", status_series, allow_duplicates=False)
        df_lift_logs["passengers_onboard"] = passengers_onboard_series
        df_lift_logs["passengers_boarded"] = passengers_boarded_series
        df_lift_logs["passengers_disembarked"] = passengers_disembarked_series

        # ---- Count Heads ----
        df_lift_logs["count_onboard"] = df_lift_logs["passengers_onboard"].apply(lambda x: len(x))
        df_lift_logs["count_boarded"] = df_lift_logs["passengers_boarded"].apply(lambda x: len(x))
        df_lift_logs["count_disembarked"] = df_lift_logs["passengers_disembarked"].apply(lambda x: len(x))
        
    return df_lift_dict


In [42]:
df_lift_elvr = elvr_logs[0]["dataframe"]
df_passenger_elvr = elvr_logs[1]["dataframe"]

df_lift_dict = get_elevator_logs_dict(df_lift_elvr, df_passenger_elvr)
df_lift_dict[1]

,lift_id,lobby_id,time,status,load,area,passengers_onboard,passengers_boarded,passengers_disembarked,count_onboard,count_boarded,count_disembarked
0,1,1,39600.0,arriving,0.0,0.00,[],[],[],0,0,0
1,1,1,39610.3,ascending,150.0,0.42,"[0, 6]","[0, 6]",[],2,2,0
2,1,3,39633.9,arriving,150.0,0.42,"[0, 6]",[],[],2,0,0
3,1,3,39643.0,descending,0.0,0.00,[],[],"[0, 6]",0,0,2
4,1,1,39666.6,arriving,0.0,0.00,[],[],[],0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
116,1,1,41944.8,arriving,0.0,0.00,[],[],[],0,0,0
117,1,1,41971.3,ascending,1350.0,3.78,"[1993, 2062, 2067, 2069, 2070, 2071, 2072, 207...","[1993, 2062, 2067, 2069, 2070, 2071, 2072, 207...",[],18,18,0
118,1,2,41988.2,arriving,1350.0,3.78,"[1993, 2062, 2067, 2069, 2070, 2071, 2072, 207...",[],[],18,0,0
119,1,2,42010.5,ascending,375.0,1.05,"[1993, 2067, 2131, 2135, 2139]",[],"[2062, 2069, 2070, 2071, 2072, 2074, 2075, 211...",5,0,13


In [171]:
def get_passenger_logs_dataframe(df_passenger_elvr, df_lift_elvr) -> pd.DataFrame:
    # ---- Fetch Passenger Dataframe ----
    df_passenger = df_passenger_elvr[["lobby_id", "destination_id", "lift_id", "time_arrived", "time_boarding", "time_destination_reached", "time_disembarked"]].copy()
    df_passenger.insert(0, "passenger_id", range(len(df_passenger)), allow_duplicates=False)
    # ---- Calculate KPI Metrics Part 1 ----
    df_passenger['wait_time'] = round(df_passenger['time_boarding'] - df_passenger['time_arrived'],1)
    df_passenger['transit_time'] = round(df_passenger['time_destination_reached'] - df_passenger['time_boarding'],1)
    df_passenger['travel_time'] = round(df_passenger['time_disembarked'] - df_passenger['time_arrived'],1)

    # ---- Create a dictionary of DataFrames for each lift_id ----
    df_lift_logs_short = df_lift_elvr[['lift_id', 'lobby_id', 'time']].copy()
    df_lift_logs_short = df_lift_logs_short.sort_values(['lift_id', 'lobby_id', 'time'])
    grouped_lift_logs = df_lift_logs_short.groupby(['lift_id', 'lobby_id'])


    # ---- Initialize New Fields ----
    time_lift_arrived_series = []
    time_lift_departed_series = []
    # ---- Iterate through each Passenger Log ----
    for passenger_log in df_passenger.itertuples(index=False):
        # Find only lift logs  where lobby_id and lift_id match the passenger's records
        df_lift_logs_matched = grouped_lift_logs.get_group((passenger_log.lift_id, passenger_log.lobby_id))
        # Find the index of the lift log that has the closest time_arrived PRIOR the passenger's time_boarding
        diff = passenger_log.time_boarding - df_lift_logs_matched["time"]
        diff = diff[diff >= 0] # Filter out negative differences
        if diff.empty:
            time_lift_arrived_series.append(pd.NA)
            time_lift_departed_series.append(pd.NA)
            continue
        else:
            index_arrival_log = diff.index[0] # Or use .idxmin()
            lift_arrival_log = df_lift_logs_matched.loc[index_arrival_log] # GroupBy doesnt change the index, so we can use .loc (Lable Indexing) to access the row
            time_lift_arrived_series.append(round(lift_arrival_log['time'],1))

            index_departure_log = df_lift_logs_matched.index.get_loc(index_arrival_log) + 1 # Switch to Positional Index from Label Index
            if index_departure_log < len(df_lift_logs_matched):
                lift_departure_log = df_lift_logs_matched.iloc[index_departure_log]
                time_lift_departed_series.append(round(lift_departure_log['time'],1))
            else:
                time_lift_departed_series.append(pd.NA)

    # ---- Insert Fields to Passenger Dataframe ----
    df_passenger.insert(3, "time_lift_arrived", time_lift_arrived_series, allow_duplicates=False)
    df_passenger.insert(6, "time_lift_departed", time_lift_departed_series, allow_duplicates=False)

    # ---- Calculat KPI Metrics Part 2----
    df_passenger["boarding_time"] = df_passenger["time_lift_departed"] - df_passenger["time_boarding"]
    df_passenger["disembarking_time"] = df_passenger["time_disembarked"] - df_passenger["time_destination_reached"]
    
    return df_passenger

In [172]:
df_lift_elvr = elvr_logs[0]["dataframe"]
df_passenger_elvr = elvr_logs[1]["dataframe"]

df_passenger_split = get_passenger_logs_dataframe(df_passenger_elvr, df_lift_elvr)
df_passenger_split

,passenger_id,lobby_id,destination_id,time_lift_arrived,lift_id,time_arrived,time_lift_departed,time_boarding,time_destination_reached,time_disembarked,wait_time,transit_time,travel_time,boarding_time,disembarking_time
0,0,1,3,39600.0,1,39601.0,39610.3,39601.1,39633.8,39635.7,0.1,32.7,34.7,9.2,1.9
1,1,1,3,39600.0,2,39601.0,39625.6,39601.2,39649.1,39651.0,0.2,47.9,50.0,24.4,1.9
2,2,1,3,39600.0,3,39601.0,39609.3,39601.3,39632.8,39634.7,0.3,31.5,33.7,8.0,1.9
3,3,1,2,39600.0,4,39602.0,39625.9,39602.1,39642.7,39644.6,0.1,40.6,42.6,23.8,1.9
4,4,1,3,39600.0,5,39602.0,39619.3,39602.2,39642.8,39644.7,0.2,40.6,42.7,17.1,1.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2155,1,3,39600.0,2,41397.0,39625.6,41976.0,42042.7,42053.0,579.0,66.7,656.0,-2350.4,10.3
2156,2156,1,3,39600.0,5,41397.0,39619.3,41976.1,42041.6,42050.7,579.1,65.5,653.7,-2356.8,9.1
2157,2157,1,3,39600.0,2,41398.0,39625.6,41976.0,42042.7,42054.2,578.0,66.7,656.2,-2350.4,11.5
2158,2158,1,3,39600.0,5,41398.0,39619.3,41976.1,42041.6,42051.9,578.1,65.5,653.9,-2356.8,10.3


In [175]:
def get_timeline_log_dataframe(df_passenger_logs: pd.DataFrame) -> pd.DataFrame:
    df_passenger_logs = pd.DataFrame(df_passenger_logs)

    start_time = math.floor(df_passenger_logs['time_arrived'].min())
    finish_time = math.ceil(df_passenger_logs['time_disembarked'].max())
    df_timeline = pd.DataFrame({'time': range(start_time, finish_time)}) # Initialize Timeline

    # ---- Initialize KPI Metrics ----
    passenger_id_register_series = []
    wait_time_register_series = []
    transit_time_register_series = []
    travel_time_register_series = []

    queue_length_series = []
    mean_wait_time_series = []
    max_wait_time_series = []
    mean_transit_time_series = []
    max_transit_time_series = []
    mean_travel_time_series = []
    max_travel_time_series = []
    # ---- Count Wait Time for each Passenger at each second ----
    for row in df_timeline.itertuples(index=False):
        # ---- Collect Passenger Logs ----
        time = row.time
        passenger_resgister = df_passenger_logs[(df_passenger_logs['time_arrived'] <= time) & (df_passenger_logs['time_boarding'] >= time)]
        # ---- Register Passenger Information ----
        passenger_id_register_series.append(passenger_resgister.index.tolist())
        wait_time_register_series.append(passenger_resgister['wait_time'].round(1).tolist())
        transit_time_register_series.append(passenger_resgister['transit_time'].round(1).tolist())
        travel_time_register_series.append(passenger_resgister['travel_time'].round(1).tolist())
        # ---- Get KPIs ----
        queue_length_series.append(len(passenger_resgister))
        mean_wait_time_series.append(round(passenger_resgister['wait_time'].mean(), 1) if not passenger_resgister.empty else 0)
        max_wait_time_series.append(round(passenger_resgister['wait_time'].max(), 1) if not passenger_resgister.empty else 0)
        mean_transit_time_series.append(round(passenger_resgister['transit_time'].mean(), 1) if not passenger_resgister.empty else 0)
        max_transit_time_series.append(round(passenger_resgister['transit_time'].max(), 1) if not passenger_resgister.empty else 0)
        mean_travel_time_series.append(round(passenger_resgister['travel_time'].mean(), 1) if not passenger_resgister.empty else 0)
        max_travel_time_series.append(round(passenger_resgister['travel_time'].max(), 1) if not passenger_resgister.empty else 0)
        
    # ---- Add KPI Metrics to Timeline ----
    df_timeline['passenger_register'] = passenger_id_register_series
    df_timeline['queue_length'] = queue_length_series
    df_timeline['mean_wait_time'] = mean_wait_time_series
    df_timeline['mean_transit_time'] = mean_transit_time_series
    df_timeline['mean_travel_time'] = mean_travel_time_series

    df_timeline['max_wait_time'] = max_wait_time_series
    df_timeline['max_transit_time'] = max_transit_time_series
    df_timeline['max_travel_time'] = max_travel_time_series

    df_timeline['wait_time_register'] = wait_time_register_series
    df_timeline['transit_time_register'] = transit_time_register_series
    df_timeline['travel_time_register'] = travel_time_register_series


    return df_timeline

In [140]:
df_timeline = get_timeline_log_dataframe(df_passenger_split)
df_timeline

,time,passenger_register,queue_length,mean_wait_time,mean_transit_time,mean_travel_time,max_wait_time,max_transit_time,max_travel_time,wait_time_register,transit_time_register,travel_time_register
0,39601,"[0, 1, 2]",3,0.2,37.4,39.5,0.3,47.9,50.0,"[0.1, 0.2, 0.3]","[32.7, 47.9, 31.5]","[34.7, 50.0, 33.7]"
1,39602,"[3, 4]",2,0.1,40.6,42.6,0.2,40.6,42.7,"[0.1, 0.2]","[40.6, 40.6]","[42.6, 42.7]"
2,39603,[5],1,0.1,32.1,34.1,0.1,32.1,34.1,[0.1],[32.1],[34.1]
3,39604,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
4,39605,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
2460,42061,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
2461,42062,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
2462,42063,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
2463,42064,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]


#### Update Passenger Logs with Timeline Data

In [178]:
def update_passenger_logs_with_timeline_data(df_passenger_logs:pd.DataFrame, df_timeline:pd.DataFrame):
    #pas_register_series = df_timeline["passenger_register"]
    queue_length_series = []
    for log in df_passenger_logs .itertuples(index=False):
        timeline_scope = df_timeline[(df_timeline["time"] >= log.time_arrived) & (df_timeline["time"] <= log.time_boarding)]
        queue_length_series.append(timeline_scope["queue_length"].max() if not timeline_scope.empty else 0)
    df_passenger_logs["queue_length"] = queue_length_series
    return df_passenger_logs

In [143]:
df_passenger_split = update_passenger_logs_with_timeline_data(df_passenger_split, df_timeline)
df_passenger_split

,passenger_id,lobby_id,destination_id,time_lift_arrived,lift_id,time_arrived,time_lift_departed,time_boarding,time_destination_reached,time_disembarked,wait_time,transit_time,travel_time,boarding_time,disembarking_time,queue_length
0,0,1,3,39600.0,1,39601.0,39610.3,39601.1,39633.8,39635.7,0.1,32.7,34.7,9.2,1.9,3
1,1,1,3,39600.0,2,39601.0,39625.6,39601.2,39649.1,39651.0,0.2,47.9,50.0,24.4,1.9,3
2,2,1,3,39600.0,3,39601.0,39609.3,39601.3,39632.8,39634.7,0.3,31.5,33.7,8.0,1.9,3
3,3,1,2,39600.0,4,39602.0,39625.9,39602.1,39642.7,39644.6,0.1,40.6,42.6,23.8,1.9,2
4,4,1,3,39600.0,5,39602.0,39619.3,39602.2,39642.8,39644.7,0.2,40.6,42.7,17.1,1.9,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,2155,1,3,39600.0,2,41397.0,39625.6,41976.0,42042.7,42053.0,579.0,66.7,656.0,-2350.4,10.3,520
2156,2156,1,3,39600.0,5,41397.0,39619.3,41976.1,42041.6,42050.7,579.1,65.5,653.7,-2356.8,9.1,520
2157,2157,1,3,39600.0,2,41398.0,39625.6,41976.0,42042.7,42054.2,578.0,66.7,656.2,-2350.4,11.5,520
2158,2158,1,3,39600.0,5,41398.0,39619.3,41976.1,42041.6,42051.9,578.1,65.5,653.9,-2356.8,10.3,520


#### Compile Timelines

In [86]:
def compile_timeline_logs(df_timelines:list[pd.DataFrame]) -> pd.DataFrame:
    # ---- Aggregate Timeline Dataframes ----
    df_compiled = pd.concat(df_timelines).groupby('time', as_index=False).agg(list)
    # ---- Drop Fields ----
    df_compiled.drop(columns=['passenger_register'], inplace=True)

    # ---- Compile KPI Metrics ----
    # ---- Insert Fields to Passenger Dataframe ----
    
    queue_length_mean = df_compiled['queue_length'].apply(lambda x: round(sum(x)/len(x),1) if len(x)!= 0 else 0)
    queue_length_max = df_compiled['queue_length'].apply(lambda x: max(x) if len(x)!= 0 else 0)
    df_compiled.drop(columns=['queue_length'], inplace=True)
    df_compiled.insert(1, "queue_length_mean", queue_length_mean, allow_duplicates=False)
    df_compiled.insert(2, "queue_length_max", queue_length_max, allow_duplicates=False)

    df_compiled['wait_time_register'] = df_compiled['wait_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    df_compiled['transit_time_register'] = df_compiled['transit_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    df_compiled['travel_time_register'] = df_compiled['travel_time_register'].apply(lambda list_of_lists: [item for sublist in list_of_lists for item in sublist])
    
    df_compiled['mean_wait_time'] = df_compiled['wait_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)
    df_compiled['mean_transit_time'] = df_compiled['transit_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)
    df_compiled['mean_travel_time'] = df_compiled['travel_time_register'].apply(lambda x: round(sum(x)/len(x), 1) if len(x)!= 0 else 0)

    df_compiled['max_wait_time'] = df_compiled['max_wait_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)
    df_compiled['max_transit_time'] = df_compiled['max_transit_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)
    df_compiled['max_travel_time'] = df_compiled['max_travel_time'].apply(lambda x: max(x) if len(x)!= 0 else 0)

    return df_compiled

In [129]:
df_timelines = []
df_passengers = []
for i in range(0,6,2):
    df_lift_elvr = elvr_logs[i]["dataframe"]
    df_passenger_elvr = elvr_logs[i+1]["dataframe"]
    df_passenger_split = get_passenger_logs_dataframe(df_passenger_elvr, df_lift_elvr)
    df_timeline = get_timeline_log_dataframe(df_passenger_split)
    df_passengers.append(df_passenger_split)
    df_timelines.append(df_timeline)

In [130]:
len(df_timelines)
df_compiled = compile_timeline_logs(df_timelines)
df_compiled.head(5)

,time,queue_length_mean,queue_length_max,mean_wait_time,mean_transit_time,mean_travel_time,max_wait_time,max_transit_time,max_travel_time,wait_time_register,transit_time_register,travel_time_register
0,39600,3.0,3,0.1,32.0,34.0,0.2,33.8,35.7,"[0.0, 0.1, 0.2]","[33.8, 29.3, 32.9]","[35.7, 31.3, 35.0]"
1,39601,1.5,3,0.2,37.4,39.5,0.3,47.9,50.0,"[0.1, 0.2, 0.3]","[32.7, 47.9, 31.5]","[34.7, 50.0, 33.7]"
2,39602,1.7,2,0.1,39.0,41.0,0.2,50.0,52.0,"[0.1, 0.2, 0.1, 0.1, 0.2]","[40.6, 40.6, 29.1, 50.0, 34.6]","[42.6, 42.7, 31.1, 52.0, 36.7]"
3,39603,1.0,2,0.1,37.2,39.2,0.2,50.4,52.4,"[0.1, 0.1, 0.2]","[32.1, 50.4, 29.0]","[34.1, 52.4, 31.1]"
4,39604,0.7,2,0.2,36.8,38.9,0.2,45.6,47.7,"[0.1, 0.2]","[28.0, 45.6]","[30.0, 47.7]"


#### Compile Passenger Logs

In [131]:
def compile_passenger_logs(df_passenger_logs: list[pd.DataFrame]) -> pd.DataFrame:
    log_list = [log.copy(deep=True) for log in df_passenger_logs]
    for i, log in enumerate(log_list):
        log["passenger_id"] = str(i) + "-" + log["passenger_id"].astype(str)
    # ---- Aggregate Passenger Dataframes ----
    df_compiled = pd.concat(log_list, axis=0).reset_index(drop=True)
    return df_compiled

In [137]:
df_passengers_compiled = compile_passenger_logs(df_passengers)
df_passengers_compiled.tail(5)

,passenger_id,lobby_id,destination_id,time_lift_arrived,lift_id,time_arrived,time_lift_departed,time_boarding,time_destination_reached,time_disembarked,wait_time,transit_time,travel_time,boarding_time,disembarking_time
6475,2-2155,1,3,39600.0,4,41397.0,39626.3,41966.9,42012.8,42026.7,569.9,45.9,629.7,-2340.6,13.9
6476,2-2156,1,3,39600.0,4,41397.0,39626.3,41966.9,42012.8,42027.9,569.9,45.9,630.9,-2340.6,15.1
6477,2-2157,1,2,39600.0,1,41398.0,39635.3,41996.4,42026.0,42030.3,598.4,29.6,632.3,-2361.1,4.3
6478,2-2158,1,3,39600.0,4,41399.0,39626.3,41966.9,42012.8,42029.1,567.9,45.9,630.1,-2340.6,16.3
6479,2-2159,1,2,39600.0,1,41399.0,39635.3,41996.4,42026.0,42031.5,597.4,29.6,632.5,-2361.1,5.5


#### Get Summary KPI

In [173]:
def get_summary_kpi(passenger_logs_dataframes:pd.DataFrame, timeline_dataframes:pd.DataFrame):
    df_pas = passenger_logs_dataframes
    df_tim = timeline_dataframes
    summary = {}
    # ---- Summary KPIs from Timeline ----
    summary["peak_time"] = df_tim.loc[df_tim['queue_length'].idxmax(), 'time']
    summary["queue_length"] = df_tim['queue_length'].max()
    summary["time_span"] = df_tim['time'].max() - df_tim['time'].iloc[0]

    # ---- Summary KPIs from Passenger Logs ----
    summary["mean_wait_time"] = round(df_pas['wait_time'].mean(),1)
    summary["mean_transit_time"] = round(df_pas['transit_time'].mean(),1)
    summary["mean_travel_time"] = round(df_pas['travel_time'].mean(),1)
    summary["max_wait_time"] = df_pas['wait_time'].max()
    summary["max_transit_time"] = df_pas['transit_time'].max()
    summary["max_travel_time"] = df_pas['travel_time'].max()
    summary["passenger_count"] = len(df_pas)

    return summary

def get_compiled_kpi(timeline_dataframes: list[pd.DataFrame], passenger_logs_dataframes: list[pd.DataFrame]) -> dict:
    df_tim = pd.concat(timeline_dataframes, axis=0).reset_index(drop=True)
    df_pas = pd.concat(passenger_logs_dataframes, axis=0).reset_index(drop=True)
    # ---- Calculate Summary KPIs ----
    summary = get_summary_kpi(df_pas, df_tim)
    timespan = 0
    passenger_count = 0
    for i in range(len(timeline_dataframes)):
        df_timeline = timeline_dataframes[i]
        df_passenger = passenger_logs_dataframes[i]
        timespan = max(timespan, df_timeline['time'].max() - df_timeline['time'].iloc[0])
        passenger_count = max(passenger_count, len(df_passenger))
    
    summary["time_span"] = timespan
    summary["passenger_count"] = passenger_count
    return summary

In [174]:
summary = get_summary_kpi(df_passenger_split, df_timeline)
summary

{'peak_time': np.int64(41330),
 'queue_length': np.int64(529),
 'mean_wait_time': np.float64(280.0),
 'mean_transit_time': np.float64(63.2),
 'mean_travel_time': np.float64(350.7),
 'max_wait_time': np.float64(1263.0),
 'max_transit_time': np.float64(91.8),
 'max_travel_time': np.float64(1308.3)}

#### Generate Logs

In [183]:
def generate_logs(elvr_logs):
    scenario_logs = []
    unique_sim_ids = list({log["simulation_id"] for log in elvr_logs})
    # ---- Iterate through each Simulation ID ----
    for sim_id in unique_sim_ids:
        # ---- Collect Log Data for current scenario ----
        matching_elvr_logs = [log for log in elvr_logs if log["simulation_id"] == sim_id]
        # ---- Skip if no logs for this simulation ID ----
        if not matching_elvr_logs: continue 
        # ---- Initialize Log Data----
        scenario_log = {}
        scenario_log["simulation_id"] = sim_id
        scenario_log["lift_count"] = matching_elvr_logs[0]["lift_count"]
        scenario_log["run_count"] = sum(1 for log in matching_elvr_logs if log["category"] == "Person")

        # ---- Initialize Table Logs ----
        lift_table_dicts = []
        passenger_tables = []
        timeline_tables = []
        summary_dicts = []
        # ---- Iterate through each Run ----
        unique_run_ids = list({log["run"] for log in matching_elvr_logs})
        for run_id in unique_run_ids:
            # ---- Collect Logs for current run ----
            elvr_logs_per_run = [log for log in matching_elvr_logs if log["run"] == run_id]
            df_passenger_elvr = None
            df_lift_elvr = None
            for log in elvr_logs_per_run:
                if log["category"] == "Person":
                    df_passenger_elvr = log["dataframe"]
                elif log["category"] == "SpatialPlot":
                    df_lift_elvr = log["dataframe"]
            # ---- Skip if no passenger or lift logs for this run ----
            if df_passenger_elvr is None or df_lift_elvr is None:
                print(f"Skipping run {run_id} for simulation {sim_id} due to missing data.")
                continue
            # ---- Log Dataframes to Scenarios ----
            lift_table_dict = get_elevator_logs_dict(df_lift_elvr, df_passenger_elvr)
            passenger_table = get_passenger_logs_dataframe(df_passenger_elvr, df_lift_elvr)
            timeline_table = get_timeline_log_dataframe(passenger_table)
            passenger_table = update_passenger_logs_with_timeline_data(passenger_table, timeline_table)
            summary_dict = get_summary_kpi(passenger_table, timeline_table)

            lift_table_dicts.append(lift_table_dict)
            passenger_tables.append(passenger_table)
            timeline_tables.append(timeline_table)
            summary_dicts.append(summary_dict)
        
        # ---- Compile Run Data ----
        timeline_compiled = compile_timeline_logs(timeline_tables)
        compiled_summary = get_compiled_kpi(timeline_tables, passenger_tables)

        # ---- Log Scenario Dataframes ----
        scenario_log["summary_dicts"] = summary_dicts
        scenario_log["lift_table_dicts"] = lift_table_dicts
        scenario_log["passenger_tables"] = passenger_tables
        scenario_log["timeline_tables"] = timeline_tables
        scenario_log["summary_compiled"] = compiled_summary
        scenario_log["timeline_compiled"] = timeline_compiled

        # ---- Update Log Data with Dataframe Information ----
        scenario_log["floor_count"] = passenger_tables[0]["lobby_id"].nunique() if passenger_tables else 0
        
        # ---- Log Scenario ----
        scenario_logs.append(scenario_log)

    return scenario_logs

In [184]:
from collections import Counter
simulation_id_counts = Counter(log["simulation_id"] for log in elvr_logs)
print(f"Scenarios Found: {simulation_id_counts}")

scenario_logs = generate_logs(elvr_logs)

Scenarios Found: Counter({'726': 20, '847': 20, '968': 20})


In [182]:
timeline = scenario_logs[0]["timeline_tables"][0]  # Example to access the first scenario's simulation ID
timeline

,time,passenger_register,queue_length,mean_wait_time,mean_transit_time,mean_travel_time,max_wait_time,max_transit_time,max_travel_time,wait_time_register,transit_time_register,travel_time_register
0,39600,[0],1,0.0,24.8,26.7,0.0,24.8,26.7,[0.0],[24.8],[26.7]
1,39601,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
2,39602,"[1, 2]",2,0.2,31.5,33.6,0.2,31.5,33.6,"[0.1, 0.2]","[31.5, 31.5]","[33.5, 33.6]"
3,39603,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
4,39604,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1877,41477,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
1878,41478,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
1879,41479,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
1880,41480,[],0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
